In [10]:
# Import Dependencies
import numpy as np
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# map drive to retrieve datasets
from google.colab import drive
drive.mount('/content/Drive')

# import dataset
law_inc_df = pd.read_csv('/content/Drive/MyDrive/Final_Project/segment_2_machine_learning/Cleaning_ML/law_inc_df.csv')
law_inc_df.head()

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).
Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal,deaths,injuries,state_year,deaths_above_below_mean
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,10,3,9,Alabama_2013,below_national_mean
1,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,1,8,8,16,Arizona_2013,below_national_mean
2,1,1,0,1,0,0,1,0,1,1,...,1,1,1,1,1,99,46,139,California_2013,below_national_mean
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,30,4,9,Colorado_2013,below_national_mean
4,1,1,1,1,1,0,0,1,1,1,...,0,0,0,0,1,82,2,13,Connecticut_2013,below_national_mean


In [3]:
# law_inc_df = law_inc_df.set_index(['state_year'])

In [4]:
# Generate our categorical variable list
law_cat = law_inc_df.dtypes[law_inc_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
law_inc_df[law_cat].nunique()

state_year                 373
deaths_above_below_mean      2
dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(law_inc_df[law_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(law_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,state_year_Alabama_2013,state_year_Alabama_2014,state_year_Alabama_2015,state_year_Alabama_2016,state_year_Alabama_2017,state_year_Alabama_2018,state_year_Alabama_2019,state_year_Alabama_2020,state_year_Alaska_2014,state_year_Alaska_2015,...,state_year_Wisconsin_2019,state_year_Wisconsin_2020,state_year_Wyoming_2014,state_year_Wyoming_2015,state_year_Wyoming_2016,state_year_Wyoming_2017,state_year_Wyoming_2018,state_year_Wyoming_2020,deaths_above_below_mean_above_national_mean,deaths_above_below_mean_below_national_mean
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
law_inc_df = law_inc_df.merge(encode_df,left_index=True, right_index=True)
law_inc_df = law_inc_df.drop(law_cat,1)
law_inc_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,violent,violenth,...,state_year_Wisconsin_2019,state_year_Wisconsin_2020,state_year_Wyoming_2014,state_year_Wyoming_2015,state_year_Wyoming_2016,state_year_Wyoming_2017,state_year_Wyoming_2018,state_year_Wyoming_2020,deaths_above_below_mean_above_national_mean,deaths_above_below_mean_below_national_mean
0,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,0,1,0,0,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,1,1,1,0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# standardize numberical variables

# Remove loan status target from features data
y = law_inc_df.deaths_above_below_mean_above_national_mean
X = law_inc_df.drop(columns=["deaths_above_below_mean_above_national_mean","deaths_above_below_mean_below_national_mean"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.947


In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
9/9 [==============================] - 2s 6ms/step - loss: 0.6432 - accuracy: 0.6559
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 0.5400 - accuracy: 0.7419
Epoch 3/50
9/9 [==============================] - 0s 4ms/step - loss: 0.4743 - accuracy: 0.7742
Epoch 4/50
9/9 [==============================] - 0s 4ms/step - loss: 0.4195 - accuracy: 0.8136
Epoch 5/50
9/9 [==============================] - 0s 4ms/step - loss: 0.3702 - accuracy: 0.8459
Epoch 6/50
9/9 [==============================] - 0s 4ms/step - loss: 0.3285 - accuracy: 0.8674
Epoch 7/50
9/9 [==============================] - 0s 4ms/step - loss: 0.2927 - accuracy: 0.9140
Epoch 8/50
9/9 [==============================] - 0s 4ms/step - loss: 0.2581 - accuracy: 0.9319
Epoch 9/50
9/9 [==============================] - 0s 5ms/step - loss: 0.2281 - accuracy: 0.9534
Epoch 10/50
9/9 [==============================] - 0s 4ms/step - loss: 0.2015 - accuracy: 0.9642
Epoch 11/50
9/9 [======================